In [29]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
import geopandas as gpd

In [7]:
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

m_1

In [12]:
# Read crimes data
crimes = pd.read_csv('./data/crimes-in-boston/crimes-in-boston/crime.csv', engine='python')
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [46]:
# Get Daytime robberies in the city
# crimes['OFFENSE_CODE_GROUP'].value_counts().head(40)
day_time_robberies = crimes[(crimes['OFFENSE_CODE_GROUP'] == 'Robbery') & (crimes["HOUR"].isin(range(17, 18)))].copy()
day_time_robberies = day_time_robberies[(day_time_robberies['Lat'].notna()) & (day_time_robberies['Long'].notna())].copy()
day_time_robberies.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
299,I182070598,311,Robbery,ROBBERY - COMMERCIAL,A15,60,NaN,2018-09-02 17:15:00,2018,9,Sunday,17,Part One,RUTHERFORD AVE,42.371673,-71.063264,"(42.37167264, -71.06326413)"
527,I182070342,381,Robbery,ROBBERY - CAR JACKING,E18,490,NaN,2018-09-01 17:05:00,2018,9,Saturday,17,Part One,CUMMINS HWY,42.276453,-71.112980,"(42.27645319, -71.11297971)"
5537,I182064868,361,Robbery,ROBBERY - OTHER,A1,111,NaN,2018-08-14 17:53:00,2018,8,Tuesday,17,Part One,AVERY ST,42.353276,-71.063458,"(42.35327561, -71.06345772)"
7203,I182063023,301,Robbery,ROBBERY - STREET,A1,112,NaN,2018-08-08 17:45:00,2018,8,Wednesday,17,Part One,AVENUE DE LAFAYETTE,42.353320,-71.061023,"(42.35331987, -71.06102314)"
8822,I182061237,311,Robbery,ROBBERY - COMMERCIAL,C11,348,NaN,2018-08-02 17:42:00,2018,8,Thursday,17,Part One,DORCHESTER AVE,42.305594,-71.058879,"(42.30559368, -71.05887861)"


In [47]:
m_2 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=13)

for idx, row in day_time_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)
m_2

In [50]:
# Create marker clusters
m_3 = folium.Map(location=[42.32, -71.0589], tiles='openstreetmap', zoom_start=13)
mc = MarkerCluster()

for idx, row in day_time_robberies.iterrows():
    mc.add_child(Marker([row['Lat'], row['Long']]))
m_3.add_child(mc)

m_3

### Bubble Maps

In [54]:
# Create base map
m_4 = folium.Map(location=[42.32, -71.0589], tiles='openstreetmap', zoom_start=13)

def color_generator(hour):
    if hour <= 12:
        return 'green'
    else:
        return 'red'

# As the data extracted has times 17 - 18, we will only get red points
for idx, row in day_time_robberies.iterrows():
    Circle(location=[row['Lat'], row['Long']], color=color_generator(row['HOUR']), radius=20).add_to(m_4)

m_4

In [55]:
# Add buuble maps as clusters
m_5 = folium.Map(location=[42.32, -71.0589], zoom_start=13, tiles='openstreetmap')

cluster = MarkerCluster()

for idx in range(0, len(day_time_robberies)):
    cluster.add_child(Circle(location=[day_time_robberies.iloc[idx]['Lat'], day_time_robberies.iloc[idx]['Long']],
                            color=color_generator(day_time_robberies.iloc[idx]['HOUR']), radius=20))
m_5.add_child(cluster)

m_5